<a href="https://colab.research.google.com/github/denisekhuu/introduction_to_pytorch/blob/main/tensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tensors: Multidimensional Arrays

Chapter 3 of Deep Learning with PyTorch:
url: https://isip.piconepress.com/courses/temple/ece_4822/resources/books/Deep-Learning-with-PyTorch.pdf

* Python lists or tuples of numbers are collections of Python objects that are individually
allocated in memory
* PyTorch tensors or NumPy
arrays, on the other hand, are views over (typically) contiguous memory blocks
* storing a 1D
tensor of 1,000,000 float numbers will require exactly 4,000,000 contiguous bytes

In [9]:

import torch 


In [ ]:
# Working with Tensors 
a = torch.ones(5)
a[1]
float(a[1])

1.0

In [ ]:
# initializing a multidimensional array
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.shape # three rows and two columns
points[1:]
points[1:, :] #All rows after the first; all columns
points[1:, 0] #All rows after the first: first column 
points[None] # Adds a dimension of size 1, just like unsqueeze
torch.squeeze(points)


tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

## Named Tensors

In [ ]:
# Named tensors
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [ ]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [ ]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels']) # Name columns
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [ ]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [ ]:
weights_aligned = weights_named.align_as(img_named) # Align weigths with naming convention
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [ ]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

## Tensor Element Types

* Numbers in Python are objects. Whereas a floating-point number might require
only, for instance, 32 bits to be represented on a computer, Python will convert
it into a full-fledged Python object with reference counting, and so on. This
operation, called boxing, is not a problem if we need to store a small number of
numbers, but allocating millions gets very inefficient
* Lists in Python are meant for sequential collections of objects. There are no operations
defined for, say, efficiently taking the dot product of two vectors, or summing vectors together. Also, Python lists have no way of optimizing the layout of their contents in memory, as they are indexable collections of pointers to Python objects
(of any kind, not just numbers). Finally, Python lists are one-dimensional, and
although we can create lists of lists, this is again very inefficient.
* The Python interpreter is slow compared to optimized, compiled code. Performing mathematical operations on large collections of numerical data can be much faster
using optimized code written in a compiled, low-level language like C.

  - torch.float32 or torch.float: 32-bit floating-point
  - torch.float64 or torch.double: 64-bit, double-precision floating-point
  - torch.float16 or torch.half: 16-bit, half-precision floating-point
  -  torch.int8: signed 8-bit integers
  - torch.uint8: unsigned 8-bit integers
  -  torch.int16 or torch.short: signed 16-bit integers
  -  torch.int32 or torch.int: signed 32-bit integers
  -  torch.int64 or torch.long: signed 64-bit integers
  - torch.bool: Boolean

In [ ]:
# Managing a tensor's dtype attribute
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

### Indexing into storage 

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage() # One dimensional storage
points_storage[0] = 2.0 # assign new value to points[0][0]
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

### Modifying stored values: In-place operations

In [ ]:
a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [ ]:
a = torch.ones(3, 2)
b = a.new_zeros(3,2)
a, b

(tensor([[1., 1.],
         [1., 1.],
         [1., 1.]]), tensor([[0., 0.],
         [0., 0.],
         [0., 0.]]))

### Tensor  metadata: size, offset, and stride 


The storage offset is the index in the storage corresponding to the first element in the tensor.
The stride is the number of elements in the storage that need to be skipped over to
obtain the next element along each dimension.
The stride is a tuple indicating the number of elements in the storage that have to be
skipped when the index is increased by 1 in each dimension.
Accessing an element i, j in a 2D tensor: 

storage_offset +
stride[0] * i + stride[1] * j 

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()
second_point.size()
points.stride()

(2, 1)

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]]) # Changing storage
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]]) # Witout changing storage
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

### Transposing without Copying

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [ ]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [ ]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.stride(), transpose_t.stride()

((20, 5, 1), (1, 5, 20))

In [ ]:
id(points.storage()) == id(points_t.storage())

True

In [ ]:
points.stride()

(2, 1)

In [ ]:
points_t.stride()

(1, 2)

### Continuous Tensors


In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t_cont = points_t.contiguous()
points_t_cont,points.is_contiguous(), points_t.is_contiguous(), points_t_cont.is_contiguous()
#points, points_t_cont, id(points.storage()) == id(points_t_cont.storage())

(tensor([[4., 5., 2.],
         [1., 3., 1.]]), True, False, True)

# Managing a tensor's device attribute

Note that the points_gpu tensor is not brought back to the CPU once the result has
been computed. Here’s what happened in this line:
1. The points tensor is copied to the GPU.
2. A new tensor is allocated on the GPU and used to store the result of the multiplication.
3. A handle to that GPU tensor is returned.


In [ ]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])

In [ ]:
points = 2 * points # Multiplication performed on CPU
points_gpu=2* points.to(device='cuda') # Multiplication performed on GPU

In [ ]:
points_gpu = points_gpu + 4 # Performed on GPU 
points_cpu = points_gpu.to(device='cpu') # Result returned to CPU

### NumPy Interoperability

While the default numeric type in PyTorch is 32-bit floating-point, for
NumPy it is 64-bit. As discussed in section 3.5.2, we usually want to use 32-bit
floating-points, so we need to make sure we have tensors of dtype torch
.float after converting

In [11]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [12]:
points = torch.from_numpy(points_np)

### Serializing tensors

In [ ]:
torch.save(points, './ourpoints.t')

In [14]:
with open('./ourpoints.t','wb') as f:
  torch.save(points, f)

In [15]:
points = torch.load('./ourpoints.t')

In [18]:
with open('./ourpoints.t','rb') as f:
  points = torch.load(f)

### Serializing to HDF5 with h5py

Every use case is unique, but we suspect needing to save tensors interoperably will be more common when introducing PyTorch into existing systems that already rely on different libraries. New projects probably won’t need to do this as often

In [20]:
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import h5py 
# Write to HDF5
f = h5py.File('./ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [25]:
# Read HDF5 File
f = h5py.File('./ourpoints.hdf5', 'r')
dset = f['coords']
last_points = dset[-2:]
last_points = torch.from_numpy(dset[-2:])
f.close()